In [62]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize
import pandas  as pd
import matplotlib.pyplot as plt

In [63]:
#Sigmoid function
def logit(theta,x):
    return 1/(1+np.exp(-1*theta.dot(x)))

In [64]:
def sigmoid_grad(theta,x):
    return (logit(theta,x)*(1-logit(theta,x)))

In [65]:
#load the data from dataset
data = loadmat('/Users/mengxiangyu/Downloads/Machine Learning Andrew Ng/ex3data1.mat')
weights = loadmat('/Users/mengxiangyu/Downloads/Machine Learning Andrew Ng/ex3weights.mat')

In [66]:
X=data['X']
Y= data['y'].ravel()
#weights for layer 2, 25 nodes in total
theta1=weights['Theta1']
#weights for layer 3, 10 nodes in total
theta2=weights['Theta2']
X=np.hstack((np.ones(X.shape[0]).reshape(5000,1),X))

In [67]:
l = 26
m = 400
n = 5000
k = 10


In [68]:
#Neural Network with Regularization
#Initial parameters
theta1_0 = np.random.uniform(-0.12,0.12,theta1.shape)
theta2_0 = np.random.uniform(-0.12,0.12,theta2.shape)

In [69]:
ini_paras = np.concatenate((theta1_0.flatten(), theta2_0.flatten()))
given_paras = np.concatenate((theta1.flatten(), theta2.flatten()))

In [70]:
cost_hist = []
#Cost function for optimization, it contains costfunction and gradient function
def CostFunction(paras,X, Y, m, n, l, k, lam):
    theta_1h = paras[:(l - 1) * (m + 1)].reshape(l - 1, m + 1)
    theta_2h = paras[(l - 1) * (m + 1):].reshape(k, l)
    Y_ = np.array(pd.get_dummies(Y.ravel()))

    z2 = theta_1h.dot(X.T)
    a2_p = np.vstack((np.ones(n), logit(theta_1h, X.T)))

    a3 = logit(theta_2h, a2_p)
    #Cost Function
    cost = -1/n*(np.sum(np.log(a3.T)*Y_+np.log(1-a3).T*(1-Y_)))+\
    (lam/(2*n))*((np.sum(np.square(theta_1h[:,1:])))+np.sum(np.square(theta_2h[:,1:])))

    #Gradient Function Back Propagation
    d3 = a3.T - Y_
    d2 = theta_2h[:, 1:].T.dot(d3.T) * logit(theta_1h, X.T)

    delta1 = d2.dot(X)
    delta2 = d3.T.dot(a2_p.T)

    theta1_ = np.c_[np.ones((theta_1h.shape[0], 1)), theta_1h[:, 1:]]
    theta2_ = np.c_[np.ones((theta_2h.shape[0], 1)), theta_2h[:, 1:]]
    theta1_grad = delta1 / n + (theta1_ * lam) / n
    theta2_grad = delta2 / n + (theta2_ * lam) / n
    
    #theta2_grad = 1 / n * d3.T.dot(a2_p.T) \
       # + lam / n * np.hstack((np.zeros(k).reshape(-1,1), theta_2h[:, 1:])) # this is the theta_2 grad reg term
    
     #Calculate the gradient for parameters in the second layer.
    #theta_delta = theta2_0[:, 1:].T.dot(a3 - Y_.T)
    
    #theta1_grad  = 1 / n * np.array([[np.sum(theta_delta[p] * logit(theta_1h, X.T)[p] * X.T[q]) 
     #    for q in range(m+1)] for p in range(l-1)]) \
     #   + lam / n * np.hstack((np.zeros(l-1).reshape(-1,1), theta_1h[:, 1:])) # this is the theta_1 grad reg term
    

    gradient = np.concatenate((theta1_grad.flatten(), theta2_grad.flatten()))
    cost_hist.append(cost)
    return cost, gradient

In [91]:
lam = 1.5

In [92]:
res = minimize(CostFunction, ini_paras, 
               method='L-BFGS-B', args=(X, Y, m, n, l, k, lam), jac=True, options={'maxiter': 100})

In [93]:
def accuracy(paras,X,Y):
    theta_1tr = paras[:(l - 1) * (m + 1)].reshape(l - 1, m + 1)
    theta_2tr = paras[(l - 1) * (m + 1):].reshape(k, l)
    
    z2 = theta_1tr.dot(X.T)
    a2_p = np.vstack((np.ones(n), logit(theta_1tr, X.T)))
    a3 = logit(theta_2tr, a2_p)
    
    return np.sum((a3.argmax(axis=0) + 1) == Y) / n
    

In [94]:
accuracy(res['x'],X,Y)

0.7018

In [95]:
accuracy(given_paras,X,Y)

0.9752